# Artificial Faces Recognition Project
----------
Importing libraries:

In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import io
import glob
import random
import shutil

Setting up our training, validation and testing data folders:

In [2]:
train_frac = 0.6
validation_frac = 0.2

reset_dataset = False

#Let's check if the folders already exist, and if not, create them. We may want to create them again with different settings too
if not os.path.isdir('real_and_fake_face/train/fake') or reset_dataset:

    #Deleting pre existing folders if they exist
    if os.path.isdir('real_and_fake_face/train/fake'):
        shutil.rmtree('real_and_fake_face/train/real')
        shutil.rmtree('real_and_fake_face/train/fake')
        shutil.rmtree('real_and_fake_face/validation/real')    
        shutil.rmtree('real_and_fake_face/validation/fake')
        shutil.rmtree('real_and_fake_face/test/real')
        shutil.rmtree('real_and_fake_face/test/fake')

    #Creating empty folders
    os.makedirs('real_and_fake_face/train/real')
    os.makedirs('real_and_fake_face/train/fake')
    os.makedirs('real_and_fake_face/validation/real')    
    os.makedirs('real_and_fake_face/validation/fake')
    os.makedirs('real_and_fake_face/test/real')
    os.makedirs('real_and_fake_face/test/fake')

    #Separating the training, validation and test data
    real_files = os.listdir('real_and_fake_face/real_complete')
    random.shuffle(real_files)
    count_real = len(real_files)
    fake_files = os.listdir('real_and_fake_face/fake_complete')
    random.shuffle(fake_files)
    count_fake = len(fake_files)

    train_real = real_files[0:int(count_real*train_frac)]
    validation_real = real_files[int(count_real*train_frac):int(count_real*train_frac)+int(count_real*validation_frac)]
    test_real = real_files[int(count_real*train_frac)+int(count_real*validation_frac)::]

    train_fake = fake_files[0:int(count_fake*train_frac)]
    validation_fake = fake_files[int(count_fake*train_frac):int(count_fake*train_frac)+int(count_fake*validation_frac)]
    test_fake = fake_files[int(count_fake*train_frac)+int(count_fake*validation_frac)::]

    #Filling the empty folders
    for file in train_real:
        src = 'real_and_fake_face/real_complete/'+file
        dst = 'real_and_fake_face/train/real/'+file
        shutil.copyfile(src,dst)
    for file in validation_real:
        src = 'real_and_fake_face/real_complete/'+file
        dst = 'real_and_fake_face/validation/real/'+file
        shutil.copyfile(src,dst)
    for file in test_real:
        src = 'real_and_fake_face/real_complete/'+file
        dst = 'real_and_fake_face/test/real/'+file
        shutil.copyfile(src,dst)
    for file in train_fake:
        src = 'real_and_fake_face/fake_complete/'+file
        dst = 'real_and_fake_face/train/fake/'+file
        shutil.copyfile(src,dst)
    for file in validation_fake:
        src = 'real_and_fake_face/fake_complete/'+file
        dst = 'real_and_fake_face/validation/fake/'+file
        shutil.copyfile(src,dst)
    for file in test_fake:
        src = 'real_and_fake_face/fake_complete/'+file
        dst = 'real_and_fake_face/test/fake/'+file
        shutil.copyfile(src,dst)


Configuring the reading and preprocessing of data:

In [3]:
train_path = 'real_and_fake_face/train'
validation_path = 'real_and_fake_face/validation'
test_path = 'real_and_fake_face/test'

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input, \
    rotation_range=10, horizontal_flip=True) \
    .flow_from_directory(directory=train_path,target_size=(224,224),classes=['real','fake'],batch_size=10)
validation_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input, \
    rotation_range=10, horizontal_flip=True) \
    .flow_from_directory(directory=validation_path,target_size=(224,224),classes=['real','fake'],batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input, \
    rotation_range=10, horizontal_flip=True) \
    .flow_from_directory(directory=test_path,target_size=(224,224),classes=['real','fake'],batch_size=10)


Found 1224 images belonging to 2 classes.
Found 408 images belonging to 2 classes.
Found 409 images belonging to 2 classes.


In [74]:
model = Sequential()
model.add(keras.layers.Conv2D(filters=32, kernel_size=(3,3),activation='relu',padding='same',input_shape=(224,224,3),\
    kernel_constraint=max_norm(0.5), bias_constraint=max_norm(0.5)))           
model.add(keras.layers.MaxPool2D(pool_size=(2,2),strides=2))
model.add(keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same',\
    kernel_constraint=max_norm(0.5), bias_constraint=max_norm(0.5)))
model.add(keras.layers.MaxPool2D(pool_size=(2,2),strides=2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=2,activation='softmax'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 56, 56, 64)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 200704)            0         
_________________________________________________________________
dense_22 (Dense)             (None, 2)                 401410    
Total params: 420,802
Trainable params: 420,802
Non-trainable params: 0
________________________________________________

In [75]:
#model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
#model.fit(x=train_batches,validation_data=validation_batches,epochs=10,verbose=2)

In [76]:
mobile = tf.keras.applications.mobilenet.MobileNet()
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [88]:

end_layers_at = -6
kept_layers = mobile.layers[end_layers_at].output
kept_layers = keras.layers.Dense(units=1024,activation='relu')(kept_layers)
kept_layers = keras.layers.BatchNormalization()(kept_layers)
kept_layers = keras.layers.Dropout(0.2)(kept_layers)
output = keras.layers.Dense(units=2,activation='softmax')(kept_layers)
model = Model(inputs=mobile.input,outputs=output)

freeze_layers_until = -1
for layer in model.layers[:freeze_layers_until]:
    layer.treinable = False
model.summary()

Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128

In [89]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x=train_batches,validation_data=validation_batches,epochs=30,verbose=2)

Epoch 1/30
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the

In [90]:
model.save('warmed_up_model')

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: warmed_up_model\assets


In [4]:
model = keras.models.load_model('warmed_up_model')
model.treinable = True
'''
for layer in model.layers:
    if 'pw' in layer.name:
        layer.trainable = True
        print(layer.name)
    else:
        layer.trainable = False
'''
model.summary()

Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128

In [5]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x=train_batches,validation_data=validation_batches,epochs=30,verbose=2)

Epoch 1/30
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the

In [7]:
model.save('fine_tuned_MobileNet')

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: fine_tuned_MobileNet\assets


In [6]:
vgg16 = tf.keras.applications.vgg16.VGG16()
vgg16.summary()

553467904/553467096 [==============================] - 460s 1us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
__________________________________________________________